In [1]:
from Explanations_Models.Custom_DT.LIME import LIME
from FeatureImportance.FI import FeatureImportance
import yaml
import pandas as pd
import argparse
import os
import sys
import numpy as np
import torch
import gym
import matplotlib
import matplotlib.pyplot as plt
import unittest
from DeepLearning_Models.utils.general import join, plot_combined
from DeepLearning_Models.ActorCritic.policy_gradient import PolicyGradient
from EnvRunner import GymRunner
import json
import warnings
from tqdm import tqdm
import torch
# Suppress all deprecation warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)
warnings.filterwarnings('ignore', category=UserWarning)

yaml.add_constructor("!join", join)
parser = argparse.ArgumentParser()

parser.add_argument("--config_filename", required=False, type=str)

config_file = open("config_envs/{}.yml".format("cartpole"))
config = yaml.load(config_file, Loader=yaml.FullLoader)
config.update(yaml.load(open("config_explanations/{}.yml".format("cartpole"), encoding="utf8"), Loader= yaml.FullLoader))
Runner = GymRunner(config)
Runner.load_weights(PATH = None)

device:  cpu


c:\Users\blav1\Scripts\BryansClaimToFame\InterpretableModelExploration\EnvRunner.py:58: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.model.baseline_network.load_state_

In [14]:
class FiniteDifferences():
    def __init__(self, model, in_size, out_size):
        self.model = model
        self.in_size = in_size
        self.out_size = out_size

    def Relevence(self, x, h = .5):
        with torch.no_grad():
            x = torch.tensor(x, dtype=torch.float32)
            jac = []
            
            for i in range(self.in_size):
                
                x[i] = x[i] + h
                
                partial = self.model.forward(x)
                x[i] = x[i] - 2*h
                
                partial = (partial - self.model.forward(x))/(2*h)
                x[i] = x[i] + h
                
                jac.append(partial.detach().numpy())
            
            jac = np.stack(jac).T
       
            return jac
FD = FiniteDifferences(Runner.model.network, 4, 2)

In [2]:
FI = FeatureImportance(config, Runner.model.network)

In [3]:
FI.Relevence(np.array([[.4,.4,.4,.4],[.1,.1,.1,.1]]), h = .5)

TypeError: get_FI() missing 1 required positional argument: 'h'

In [6]:
FI = Relevence(config, Runner.model.network) 

In [7]:
out, fi = FI.Relevence(X)

In [5]:
fi.shape

(60, 2, 4)

In [6]:
A = torch.tensor(fi[0].copy())

In [7]:
vals, vecs = torch.linalg.eig(A.T@A)


In [8]:
vecs.real

tensor([[-0.4092,  0.0104, -0.7310, -0.7310],
        [-0.4091, -0.6953,  0.0740,  0.0740],
        [-0.4367, -0.3493,  0.2489,  0.2489],
        [-0.6889,  0.6281,  0.2324,  0.2324]])

In [9]:
vals.real

tensor([ 1.1882e+00,  9.1271e-03, -1.0320e-09, -1.0320e-09])

In [12]:
torch.unsqueeze(vals.real,1) * vecs.real

tensor([[-4.8614e-01,  1.2305e-02, -8.6848e-01, -8.6848e-01],
        [-3.7336e-03, -6.3459e-03,  6.7553e-04,  6.7553e-04],
        [ 4.5062e-10,  3.6043e-10, -2.5689e-10, -2.5689e-10],
        [ 7.1092e-10, -6.4817e-10, -2.3983e-10, -2.3983e-10]])

In [11]:
torch.sum(torch.unsqueeze(vals.real,1) * vecs.real, dim=0)

tensor([-0.4899,  0.0060, -0.8678, -0.8678])

In [39]:
np.array( [-0.2113,  0.7856, -0.4802, -0.5657])*   1.5070e-01

array([-0.03184291,  0.11838992, -0.07236614, -0.08525099])

In [20]:
columns = pd.MultiIndex.from_product([config["picture"]["class_names"], config["picture"]["labels"]], names=['OutLogit', 'InLogit'])
fi = np.abs(fi)
# Reshape the data to (60, 8) to match the multi-level column structurel
reshaped_data = fi.reshape(fi.shape[0], -1)

# Create the DataFrame
df = pd.DataFrame(reshaped_data, columns=columns)

In [28]:
for i in config["picture"]["class_names"]:
    print(df[i].mean())

InLogit
x            0.257444
vel          0.273353
angle        0.266402
angle_vel    0.264963
dtype: float32
InLogit
x            0.344789
vel          0.240145
angle        0.278599
angle_vel    0.295341
dtype: float32


In [31]:
type(df)

pandas.core.frame.DataFrame

In [27]:
d

OutLogit      left                                   right            \
InLogit          x       vel     angle angle_vel         x       vel   
0         0.290158  0.280733  0.282981  0.283786  0.431109  0.415831   
1         0.224832  0.225738  0.214128  0.221140  0.134304  0.049049   
2         0.167023  0.291283  0.189150  0.300101  0.187694  0.242353   
3         0.151415  0.180867  0.177579  0.182678  0.042678  0.026636   
4         0.040110  0.142284  0.145554  0.154019  0.015413  0.068957   
5         0.354664  0.407079  0.352071  0.311107  0.784774  0.336721   
6         0.036103  0.047120  0.045803  0.048404  0.123737  0.083060   
7         0.058699  0.078819  0.080805  0.079209  0.143149  0.065075   
8         0.202371  0.189437  0.191994  0.205764  0.349770  0.328939   
9         0.374310  0.389922  0.361915  0.335573  0.293567  0.232608   
10        0.408282  0.380004  0.322395  0.276485  0.826780  0.308336   
11        0.142899  0.182179  0.188435  0.202131  0.075188  0.078876   
12        0.296877  0.312056  0.318595  0.261509  0.245088  0.138506   
13        0.333207  0.461922  0.404453  0.231132  0.637112  0.392356   
14        0.280056  0.354346  0.276294  0.236178  0.563928  0.319455   
15        0.197630  0.165182  0.191269  0.409909  0.094588  0.106253   
16        0.480013  0.374098  0.311747  0.256800  0.902182  0.292739   
17        0.386286  0.486092  0.423398  0.196390  0.689567  0.405059   
18        0.030055  0.039496  0.038499  0.040463  0.132283  0.089124   
19        0.053461  0.053108  0.052180  0.066941  0.095609  0.113899   
20        0.493741  0.539174  0.539409  0.541474  0.631328  0.433982   
21        0.144409  0.147586  0.138870  0.121026  0.006511  0.008012   
22        0.428414  0.556810  0.439057  0.602706  0.330261  0.345878   
23        0.423659  0.415745  0.417302  0.369360  0.550128  0.535056   
24        0.188252  0.189023  0.181713  0.150371  0.053481  0.030199   
25        0.018689  0.018045  0.017920  0.018409  0.170213  0.126460   
26        0.327807  0.318886  0.321039  0.312367  0.465397  0.450504   
27        0.016189  0.020225  0.018049  0.020012  0.151084  0.105779   
28        0.362216  0.408310  0.398837  0.364271  0.502354  0.430248   
29        0.100259  0.156301  0.155428  0.161126  0.004259  0.003178   
30        0.576121  0.367876  0.298732  0.234561  1.006526  0.274317   
31        0.535262  0.528764  0.528943  0.472489  0.619418  0.415905   
32        0.220965  0.255610  0.241026  0.192717  0.377974  0.250133   
33        0.152084  0.056320  0.138272  0.152434  0.014884  0.000110   
34        0.090322  0.154204  0.153705  0.159308  0.009668  0.007744   
35        0.088988  0.093518  0.091027  0.085524  0.065053  0.079012   
36        0.018836  0.103709  0.103955  0.109925  0.039241  0.071525   
37        0.395638  0.338151  0.397271  0.726733  0.336932  0.386389   
38        0.190712  0.433676  0.384755  0.292353  0.477020  0.393612   
39        0.095980  0.104169  0.105910  0.103394  0.102262  0.039824   
40        0.167112  0.194684  0.180311  0.034660  0.028557  0.036738   
41        0.108926  0.461308  0.415352  0.324192  0.376372  0.428965   
42        0.278940  0.220116  0.298537  0.036890  0.747365  0.090644   
43        0.069373  0.094942  0.097172  0.095433  0.118764  0.053927   
44        0.362100  0.353541  0.355547  0.335513  0.496133  0.481295   
45        0.106214  0.082594  0.096465  0.386206  0.017853  0.019100   
46        0.273997  0.323507  0.310458  0.336895  0.416798  0.366588   
47        0.154557  0.108348  0.108382  0.104733  0.084386  0.020946   
48        0.332654  0.330584  0.323417  0.266225  0.473209  0.462488   
49        0.072449  0.079631  0.076283  0.077561  0.062422  0.066240   
50        0.348340  0.422238  0.366980  0.324895  0.786573  0.348417   
51        0.238623  0.255060  0.245984  0.242713  0.129583  0.076676   
52        0.474902  0.415400  0.474366  0.813090  0.442877  0.517350   
53        0.763157  0.483363  0.681225  

In [39]:
print(df.mean())
print(out_pd.var())

OutLogit  InLogit  
left      x            0.257444
          vel          0.273353
          angle        0.266402
          angle_vel    0.264963
right     x            0.344789
          vel          0.240145
          angle        0.278599
          angle_vel    0.295341
dtype: float32
left     1.402645
right    1.508201
dtype: float32


In [44]:
tmp = None
for i in list(out_pd.keys()):
    if tmp == None:
        tmp = df[i].mean().to_numpy() * out_pd[i].var()
    else:
        tmp += df[i].mean().to_numpy() * out_pd[i].var()

[0.25744355 0.27335274 0.2664024  0.26496324] 1.4026452
[0.34478876 0.24014513 0.27859893 0.2953409 ] 1.5082008


In [60]:
tmp = []
for i in list(out_pd.keys()):
    if len(tmp) == 0:
        tmp = df[i].mean().to_numpy() * out_pd[i].var()
    else:
        tmp += df[i].mean().to_numpy() * out_pd[i].var()
print(tmp)
print(np.expand_dims(tmp, axis = 0))
tmp = pd.DataFrame(np.expand_dims(tmp, axis = 0), columns=list(df[out_pd.keys()[0]].keys()))
print(tmp)

[0.8811126  0.745604   0.79385114 0.81708276]
[[0.8811126  0.745604   0.79385114 0.81708276]]
          x       vel     angle  angle_vel
0  0.881113  0.745604  0.793851   0.817083


In [56]:
list(df[out_pd.keys()[0]].keys())

['x', 'vel', 'angle', 'angle_vel']

In [69]:
for i in out_pd.keys():
    print(df[i].mean().to_numpy())

[0.25744355 0.27335274 0.2664024  0.26496324]
[0.34478876 0.24014513 0.27859893 0.2953409 ]


In [74]:
np.stack([df["left"].mean().to_numpy(), df["right"].mean().to_numpy()]).max(axis=0)

array([[0.25744355, 0.27335274, 0.2664024 , 0.26496324],
       [0.34478876, 0.24014513, 0.27859893, 0.2953409 ]], dtype=float32)

In [75]:
np.stack([df["left"].mean().to_numpy(), df["right"].mean().to_numpy()]).max(axis=0)

array([0.34478876, 0.27335274, 0.27859893, 0.2953409 ], dtype=float32)

In [76]:
def Policy_Smoothing_Max_Weighting(FI, out_logits):
    tmp = []
    for i in list(out_logits.keys()):
        tmp.append(FI[i].mean().to_numpy())
    tmp = np.stack(tmp).max(axis=0)
    tmp = pd.DataFrame(np.expand_dims(tmp, axis=0), columns = list(FI[out_logits.keys()[0]].keys()))
    return tmp

Policy_Smoothing_Max_Weighting(df, out_pd)

,x,vel,angle,angle_vel
0,0.344789,0.273353,0.278599,0.295341


In [77]:
def Policy_Smoothing_Var_Weighting(FI, out_logits):
    smoothed_fi = FI.mean()
    output_variance = out_logits.var()
    tmp = []
    for i in list(out_logits.keys()):
        if len(tmp) == 0:
            tmp = FI[i].mean().to_numpy() * out_logits[i].var()
        else:
            tmp += FI[i].mean().to_numpy() * out_logits[i].var()
    tmp = pd.DataFrame(np.expand_dims(tmp, axis=0), columns = list(FI[out_logits.keys()[0]].keys()))
    return tmp
Policy_Smoothing_Var_Weighting(df, out_pd)

,x,vel,angle,angle_vel
0,0.881113,0.745604,0.793851,0.817083


In [79]:
out_pd.idxmax(axis=1)

0     right
1      left
2     right
3      left
4      left
5     right
6     right
7     right
8     right
9      left
10    right
11     left
12     left
13    right
14    right
15     left
16    right
17    right
18    right
19    right
20     left
21     left
22     left
23    right
24     left
25    right
26    right
27    right
28    right
29     left
30    right
31     left
32    right
33     left
34     left
35     left
36     left
37     left
38    right
39     left
40     left
41    right
42    right
43     left
44    right
45     left
46    right
47     left
48    right
49     left
50    right
51     left
52     left
53     left
54    right
55     left
56    right
57    right
58     left
59    right
dtype: object

In [84]:
rows = df.shape[0]  # Number of rows
high_labels = df.columns.get_level_values(0).unique()  # Unique high-level labels
sub_labels = df.columns.get_level_values(1).unique()   # Unique sub-labels

# Step 2: Convert DataFrame to NumPy array and reshape
# First, rearrange columns to match the desired order: (rows, high-labels, sub-labels)
FI_np = df.to_numpy().reshape(rows, len(high_labels), len(sub_labels))

In [97]:
FI_np.shape

(60, 2, 4)

In [96]:
np.argmax(out_pd.to_numpy(), axis=1)

array([1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0,
       0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0,
       1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1], dtype=int64)

In [107]:
FI_np[np.arange(FI_np.shape[0]),np.argmax(out_pd.to_numpy(), axis=1)].mean(axis=0)

array([0.3680011 , 0.28244072, 0.32482272, 0.39116088], dtype=float32)

In [86]:
df

OutLogit      left                                   right            \
InLogit          x       vel     angle angle_vel         x       vel   
0         0.290158  0.280733  0.282981  0.283786  0.431109  0.415831   
1         0.224832  0.225738  0.214128  0.221140  0.134304  0.049049   
2         0.167023  0.291283  0.189150  0.300101  0.187694  0.242353   
3         0.151415  0.180867  0.177579  0.182678  0.042678  0.026636   
4         0.040110  0.142284  0.145554  0.154019  0.015413  0.068957   
5         0.354664  0.407079  0.352071  0.311107  0.784774  0.336721   
6         0.036103  0.047120  0.045803  0.048404  0.123737  0.083060   
7         0.058699  0.078819  0.080805  0.079209  0.143149  0.065075   
8         0.202371  0.189437  0.191994  0.205764  0.349770  0.328939   
9         0.374310  0.389922  0.361915  0.335573  0.293567  0.232608   
10        0.408282  0.380004  0.322395  0.276485  0.826780  0.308336   
11        0.142899  0.182179  0.188435  0.202131  0.075188  0.078876   
12        0.296877  0.312056  0.318595  0.261509  0.245088  0.138506   
13        0.333207  0.461922  0.404453  0.231132  0.637112  0.392356   
14        0.280056  0.354346  0.276294  0.236178  0.563928  0.319455   
15        0.197630  0.165182  0.191269  0.409909  0.094588  0.106253   
16        0.480013  0.374098  0.311747  0.256800  0.902182  0.292739   
17        0.386286  0.486092  0.423398  0.196390  0.689567  0.405059   
18        0.030055  0.039496  0.038499  0.040463  0.132283  0.089124   
19        0.053461  0.053108  0.052180  0.066941  0.095609  0.113899   
20        0.493741  0.539174  0.539409  0.541474  0.631328  0.433982   
21        0.144409  0.147586  0.138870  0.121026  0.006511  0.008012   
22        0.428414  0.556810  0.439057  0.602706  0.330261  0.345878   
23        0.423659  0.415745  0.417302  0.369360  0.550128  0.535056   
24        0.188252  0.189023  0.181713  0.150371  0.053481  0.030199   
25        0.018689  0.018045  0.017920  0.018409  0.170213  0.126460   
26        0.327807  0.318886  0.321039  0.312367  0.465397  0.450504   
27        0.016189  0.020225  0.018049  0.020012  0.151084  0.105779   
28        0.362216  0.408310  0.398837  0.364271  0.502354  0.430248   
29        0.100259  0.156301  0.155428  0.161126  0.004259  0.003178   
30        0.576121  0.367876  0.298732  0.234561  1.006526  0.274317   
31        0.535262  0.528764  0.528943  0.472489  0.619418  0.415905   
32        0.220965  0.255610  0.241026  0.192717  0.377974  0.250133   
33        0.152084  0.056320  0.138272  0.152434  0.014884  0.000110   
34        0.090322  0.154204  0.153705  0.159308  0.009668  0.007744   
35        0.088988  0.093518  0.091027  0.085524  0.065053  0.079012   
36        0.018836  0.103709  0.103955  0.109925  0.039241  0.071525   
37        0.395638  0.338151  0.397271  0.726733  0.336932  0.386389   
38        0.190712  0.433676  0.384755  0.292353  0.477020  0.393612   
39        0.095980  0.104169  0.105910  0.103394  0.102262  0.039824   
40        0.167112  0.194684  0.180311  0.034660  0.028557  0.036738   
41        0.108926  0.461308  0.415352  0.324192  0.376372  0.428965   
42        0.278940  0.220116  0.298537  0.036890  0.747365  0.090644   
43        0.069373  0.094942  0.097172  0.095433  0.118764  0.053927   
44        0.362100  0.353541  0.355547  0.335513  0.496133  0.481295   
45        0.106214  0.082594  0.096465  0.386206  0.017853  0.019100   
46        0.273997  0.323507  0.310458  0.336895  0.416798  0.366588   
47        0.154557  0.108348  0.108382  0.104733  0.084386  0.020946   
48        0.332654  0.330584  0.323417  0.266225  0.473209  0.462488   
49        0.072449  0.079631  0.076283  0.077561  0.062422  0.066240   
50        0.348340  0.422238  0.366980  0.324895  0.786573  0.348417   
51        0.238623  0.255060  0.245984  0.242713  0.129583  0.076676   
52        0.474902  0.415400  0.474366  0.813090  0.442877  0.517350   
53        0.763157  0.483363  0.681225  

In [110]:
def Classification_forgiveness(FI, out_logits):
    rows = FI.shape[0] 
    high_labels = FI.columns.get_level_values(0).unique()  
    sub_labels = FI.columns.get_level_values(1).unique() 

    
    FI_np = FI.to_numpy().reshape(rows, len(high_labels), len(sub_labels))
    tmp = FI_np[np.arange(FI_np.shape[0]),np.argmax(out_logits.to_numpy(), axis=1)].mean(axis=0)
    return pd.DataFrame(np.expand_dims(tmp, axis=0), columns = list(FI[out_logits.keys()[0]].keys()))

Classification_forgiveness(df, out_pd)

,x,vel,angle,angle_vel
0,0.368001,0.282441,0.324823,0.391161


In [109]:
df

OutLogit      left                                   right            \
InLogit          x       vel     angle angle_vel         x       vel   
0         0.290158  0.280733  0.282981  0.283786  0.431109  0.415831   
1         0.224832  0.225738  0.214128  0.221140  0.134304  0.049049   
2         0.167023  0.291283  0.189150  0.300101  0.187694  0.242353   
3         0.151415  0.180867  0.177579  0.182678  0.042678  0.026636   
4         0.040110  0.142284  0.145554  0.154019  0.015413  0.068957   
5         0.354664  0.407079  0.352071  0.311107  0.784774  0.336721   
6         0.036103  0.047120  0.045803  0.048404  0.123737  0.083060   
7         0.058699  0.078819  0.080805  0.079209  0.143149  0.065075   
8         0.202371  0.189437  0.191994  0.205764  0.349770  0.328939   
9         0.374310  0.389922  0.361915  0.335573  0.293567  0.232608   
10        0.408282  0.380004  0.322395  0.276485  0.826780  0.308336   
11        0.142899  0.182179  0.188435  0.202131  0.075188  0.078876   
12        0.296877  0.312056  0.318595  0.261509  0.245088  0.138506   
13        0.333207  0.461922  0.404453  0.231132  0.637112  0.392356   
14        0.280056  0.354346  0.276294  0.236178  0.563928  0.319455   
15        0.197630  0.165182  0.191269  0.409909  0.094588  0.106253   
16        0.480013  0.374098  0.311747  0.256800  0.902182  0.292739   
17        0.386286  0.486092  0.423398  0.196390  0.689567  0.405059   
18        0.030055  0.039496  0.038499  0.040463  0.132283  0.089124   
19        0.053461  0.053108  0.052180  0.066941  0.095609  0.113899   
20        0.493741  0.539174  0.539409  0.541474  0.631328  0.433982   
21        0.144409  0.147586  0.138870  0.121026  0.006511  0.008012   
22        0.428414  0.556810  0.439057  0.602706  0.330261  0.345878   
23        0.423659  0.415745  0.417302  0.369360  0.550128  0.535056   
24        0.188252  0.189023  0.181713  0.150371  0.053481  0.030199   
25        0.018689  0.018045  0.017920  0.018409  0.170213  0.126460   
26        0.327807  0.318886  0.321039  0.312367  0.465397  0.450504   
27        0.016189  0.020225  0.018049  0.020012  0.151084  0.105779   
28        0.362216  0.408310  0.398837  0.364271  0.502354  0.430248   
29        0.100259  0.156301  0.155428  0.161126  0.004259  0.003178   
30        0.576121  0.367876  0.298732  0.234561  1.006526  0.274317   
31        0.535262  0.528764  0.528943  0.472489  0.619418  0.415905   
32        0.220965  0.255610  0.241026  0.192717  0.377974  0.250133   
33        0.152084  0.056320  0.138272  0.152434  0.014884  0.000110   
34        0.090322  0.154204  0.153705  0.159308  0.009668  0.007744   
35        0.088988  0.093518  0.091027  0.085524  0.065053  0.079012   
36        0.018836  0.103709  0.103955  0.109925  0.039241  0.071525   
37        0.395638  0.338151  0.397271  0.726733  0.336932  0.386389   
38        0.190712  0.433676  0.384755  0.292353  0.477020  0.393612   
39        0.095980  0.104169  0.105910  0.103394  0.102262  0.039824   
40        0.167112  0.194684  0.180311  0.034660  0.028557  0.036738   
41        0.108926  0.461308  0.415352  0.324192  0.376372  0.428965   
42        0.278940  0.220116  0.298537  0.036890  0.747365  0.090644   
43        0.069373  0.094942  0.097172  0.095433  0.118764  0.053927   
44        0.362100  0.353541  0.355547  0.335513  0.496133  0.481295   
45        0.106214  0.082594  0.096465  0.386206  0.017853  0.019100   
46        0.273997  0.323507  0.310458  0.336895  0.416798  0.366588   
47        0.154557  0.108348  0.108382  0.104733  0.084386  0.020946   
48        0.332654  0.330584  0.323417  0.266225  0.473209  0.462488   
49        0.072449  0.079631  0.076283  0.077561  0.062422  0.066240   
50        0.348340  0.422238  0.366980  0.324895  0.786573  0.348417   
51        0.238623  0.255060  0.245984  0.242713  0.129583  0.076676   
52        0.474902  0.415400  0.474366  0.813090  0.442877  0.517350   
53        0.763157  0.483363  0.681225  

In [9]:

def PCA_Ish(FI, out_logits):
    tmp = FI.mean()
    arr = []
    for i in out_logits.keys():
        arr.append(tmp[i].to_numpy())
    tmp = np.stack(arr)
    print(tmp)
    A = torch.tensor(tmp, dtype=torch.float64)
    print(A)
    print(A.T@A)
    vals, vecs = torch.linalg.eigh(A.T@A)
    #print(torch.linalg.eig(A))
    print(vals.real, vecs.real)
    weighted_FI = torch.unsqueeze(vals.real,1) * vecs.real
    return torch.sum(weighted_FI, dim=0).detach().numpy()
PCA_Ish(df, out)

[[0.3424595  0.2609292  0.29197863 0.34361637]
 [0.4779225  0.18795358 0.27932766 0.3956685 ]]
tensor([[0.3425, 0.2609, 0.2920, 0.3436],
        [0.4779, 0.1880, 0.2793, 0.3957]], dtype=torch.float64)
tensor([[0.3457, 0.1792, 0.2335, 0.3068],
        [0.1792, 0.1034, 0.1287, 0.1640],
        [0.2335, 0.1287, 0.1633, 0.2108],
        [0.3068, 0.1640, 0.2108, 0.2746]], dtype=torch.float64)
tensor([3.4908e-18, 3.2071e-17, 9.7948e-03, 8.7721e-01], dtype=torch.float64) tensor([[-0.4579,  0.2155, -0.5948, -0.6246],
        [-0.2180,  0.5631,  0.7234, -0.3347],
        [-0.2417, -0.7970,  0.3487, -0.4299],
        [ 0.8273,  0.0349, -0.0367, -0.5595]], dtype=torch.float64)


array([ 0.72332287,  0.02277997, -0.02877742, -0.49501747])

In [3]:
A = torch.tensor([[0.1985, 0.1411, 0.1703, 0.1949],
                  [0.1411, 0.1125, 0.1283, 0.1434],
                  [0.1703, 0.1283, 0.1504, 0.1701],
                  [0.1949, 0.1434, 0.1701, 0.1933]], dtype=torch.float64)
v = torch.tensor([[-0.4092,  0.0104, -0.7310, -0.7310],
        [-0.4091, -0.6953,  0.0740,  0.0740],
        [-0.4367, -0.3493,  0.2489,  0.2489],
        [-0.6889,  0.6281,  0.2324,  0.2324]], dtype=torch.float64)

gammer = torch.tensor([ 1.1882e+00,  9.1271e-03, -1.0320e-09, -1.0320e-09], dtype=torch.float64)

In [4]:
torch.linalg.eigh(A)

torch.return_types.linalg_eigh(
eigenvalues=tensor([-8.5125e-06,  4.4670e-07,  8.4232e-03,  6.4628e-01],
       dtype=torch.float64),
eigenvectors=tensor([[ 0.0301,  0.5717, -0.6083, -0.5498],
        [ 0.3026,  0.4021,  0.7617, -0.4081],
        [-0.8270, -0.2210,  0.1870, -0.4819],
        [ 0.4729, -0.6802, -0.1216, -0.5467]], dtype=torch.float64))

In [37]:
A@v[0]

tensor([-0.3467, -0.2552, -0.3026, -0.3439], dtype=torch.float64)

In [35]:
torch.unsqueeze(gammer, dim=0)*v

tensor([[-4.8621e-01,  9.4922e-05,  7.5439e-10,  7.5439e-10],
        [-4.8609e-01, -6.3461e-03, -7.6368e-11, -7.6368e-11],
        [-5.1889e-01, -3.1881e-03, -2.5686e-10, -2.5686e-10],
        [-8.1855e-01,  5.7327e-03, -2.3984e-10, -2.3984e-10]],
       dtype=torch.float64)

In [39]:
A =  np.array([[0.1985, 0.1411, 0.1703, 0.1949],
        [0.1411, 0.1125, 0.1283, 0.1434],
        [0.1703, 0.1283, 0.1504, 0.1701],
        [0.1949, 0.1434, 0.1701, 0.1933]],dtype = np.float64)
np.linalg.eigh(A)

: 

In [8]:
columns = pd.MultiIndex.from_product([config["picture"]["class_names"], config["picture"]["labels"]], names=['OutLogit', 'InLogit'])
fi = np.abs(fi)
# Reshape the data to (60, 8) to match the multi-level column structurel
reshaped_data = fi.reshape(fi.shape[0], -1)

# Create the DataFrame
df = pd.DataFrame(reshaped_data, columns=columns)
out = pd.DataFrame(out, columns=config["picture"]["class_names"])